In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
save_dir = Path("../../data-cumberland/insitu")
save_dir.mkdir(exist_ok=True)

# ResOpsUS
Save it with name of reservoir and consistent column names.

In [12]:
data_directory = Path("../../global_data/ResOpsUS/time_series_all")
reservoir_attributes_fn = Path("../../global_data/ResOpsUS/attributes/reservoir_attributes.csv")
reservoirs_fn = Path("../../data-cumberland/cumberland-stations/cumberland-resops-stations.csv")

resops_attributes = pd.read_csv(reservoir_attributes_fn)
resops_attributes['DAM_NAME'] = resops_attributes['DAM_NAME'].str.replace(' ', '_')

reservoirs = pd.read_csv(reservoirs_fn)

for reservoir in reservoirs['name']:
    reservoir_id = int(resops_attributes[resops_attributes['DAM_NAME']==reservoir]['DAM_ID'])
    print(reservoir, reservoir_id)

    fn = data_directory / f"ResOpsUS_{reservoir_id}.csv"

    df = pd.read_csv(fn, parse_dates=['date'])

    save_fn = save_dir / f"{reservoir}.csv"
    df.to_csv(save_fn, index=False)

Barkley_Dam 1752
Laurel_Dam 1754
Wolf_Creek 1756
Dale_Hollow_Dam 1762
Old_Hickory_Dam 1768
J._Percy_Priest_Dam 1775
Center_Hill_Dam 1777
Great_Falls 1787
Cordell_Hull_Dam 7318


# USGS

The naming convention of USGS gages in RAT can be tricky because the name gets truncated to 5 characters. There may be multiple 

In [13]:
data_directory = Path("../../data-cumberland/usgs_insitu/")
gage_attributes_fn = Path("../../data-cumberland/usgs_gage_locations/usgs_gages_with_assigned_names.csv")

gage_attributes = pd.read_csv(gage_attributes_fn, dtype={'site_no': str})[['site_no', 'assigned_name']]

for i, row in gage_attributes.iterrows():
    fn = data_directory / f"{row.site_no}.csv"
    if fn.exists():
        save_fn = save_dir / f"{row.assigned_name}.csv"

        df = pd.read_csv(fn, parse_dates=['datetime']).rename({
                'datetime': 'date',
                '00060_Mean': 'inflow'
            }, axis=1)[['date', 'inflow']]
        df['date'] = df.date.dt.date
        df.loc[df['inflow'] < 0, 'inflo'] = np.nan
        df.tail()

        df.to_csv(save_fn, index=False)